Please prepare your LLM and the RAG and import reqyuired libraries in the bellow cell(s) before the evaluation.

In [1]:
!pip3 install -q -U bitsandbytes==0.42.0 peft==0.8.2 trl==0.7.10 accelerate==0.27.1 datasets==2.17.0 transformers==4.38.1 langchain sentence-transformers chromadb langchainhub

In [2]:
!pip install langchain-community langchain-core

In [3]:
!pip install --upgrade langchain

In [4]:
!pip install shutup

In [5]:
import shutup;
shutup.please()

In [6]:
import os
import warnings
from google.colab import userdata

# Set up the Hugging Face Hub API token
os.environ["HUGGINGFACEHUB_API_TOKEN"] = userdata.get('HF_TOKEN1')
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [7]:
from langchain_community.llms import HuggingFaceEndpoint
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

# Define the repository ID for the Gemma 2b model
repo_id = "google/gemma-2b-it"

# Set up a Hugging Face Endpoint for Gemma 2b model
llm = HuggingFaceEndpoint(
    repo_id=repo_id, max_length=1024, temperature=0.1
)

                    max_length was transferred to model_kwargs.
                    Please make sure that max_length is what you intended.


The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


# Models Evaluation:
**1- RAGAS:**
The folowing evaluation is based on[ DeepEval RAGAS](https://docs.confident-ai.com/docs/metrics-ragas) metrics which evaluate the RAG performance

* RAGAS Answer Relevancy Metric
* RAGAS Faithfulness Metric
* RAGAS Contextual PrecisionMetric
* RAGAS ContextualRecall Metric


Feed the below link of the [unipd how to apply webpage](https://www.unipd.it/en/how-apply) to your LLM RAG pipeline so it can extract the data from it.

In [8]:
Rag_input_data = "https://www.unipd.it/en/how-apply"

In [9]:
# Load a document
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader(Rag_input_data)
data = loader.load()
#print(data)

In [11]:
from langchain_community.document_loaders import TextLoader
from langchain_community.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings,
)
from langchain_community.vectorstores import Chroma
from langchain_text_splitters import CharacterTextSplitter


# split it into chunks
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(data)

# create the open-source embedding function
embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

# load it into Chroma
db = Chroma.from_documents(docs, embedding_function)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
from langchain import hub
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain.chains import RetrievalQA

retriever = db.as_retriever(search_type="mmr", search_kwargs={'k': 4, 'fetch_k': 20})
prompt = hub.pull("rlm/rag-prompt")

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
)

Please  use the Questions bellow as the prompt to your LLM and then append its outputs to the *llm_outputs* array of string

**Note: Do not change the questions bellow at all, we want to ask the same questions from different LLMs we are working on, so we can make a correct comparison**

In [ ]:
q1= "Can I apply for the admission of the university of Padua if I am a non-italian student?"

# Ask the question from your LLM and then append its output to the below array

llm_outputs = []
result = rag_chain.invoke(q1)
llm_outputs.append(result)# append the answer of your llm to the 1st question as a string here
result

' No, the context does not provide any information about the possibility of applying for the admission of the University of Padua for non-Italian students.'

In [ ]:
q2 = "Are the courses provided by the university of Padua taught only in Italian?"
result = rag_chain.invoke(q2)
llm_outputs.append(result)# append the answer of your llm to the 2n question as a string here
result

' No, the context does not specify that the courses provided by the university of Padua are taught only in Italian.'

In [ ]:
q3 = "I missed the deadlines for submitting application for the first semester. Can I enroll and submit the application for the second (winter) semester?"
result = rag_chain.invoke(q3)
llm_outputs.append(result)# append the answer of your llm to the 3rd question as a string here
result

' No, the context does not allow you to enroll and submit the application for the second (winter) semester.'

In [ ]:
q4 = "I want to apply for the bachelor's degree in biology, how to know if I am eligible?"
result = rag_chain.invoke(q4)
llm_outputs.append(result)# append the answer of your llm to the 4th question as a string here
result

' Applicants need to have a minimum of 12 years of schooling and pass all the scheduled exams to be eligible for admission to a Bachelor’s degree. Applicants can consult a country-based list of qualifications to verify their entry qualification.'

In [ ]:
q5 = "The duration of diploma that I obtained is 11 years in my country, does that mean that I can not apply for the admission of the bachelor's degree program in biology of the university of Padua?"
result = rag_chain.invoke(q5)
llm_outputs.append(result)# append the answer of your llm to the 5th question as a string here
result

" No, the context does not mention that the 11-year diploma means that the applicant cannot apply for the admission of the bachelor's degree program in biology of the University of Padua."

In [ ]:
q6 ="How can I make sure if my previous university which is not Italian is qualified in order to apply for the master's degree programs?"
result = rag_chain.invoke(q6)
llm_outputs.append(result)# append the answer of your llm to the 6th question as a string here
result

' Applicants should check the country-based list of qualifications to verify if their entry qualification is suitable to access their degree programme of interest.'

In [ ]:
q7 = "Can I apply for just single course units not the whole degree programs of the University of Padua?"
result = rag_chain.invoke(q7)
llm_outputs.append(result)# append the answer of your llm to the 7th question as a string here
result

' No, the context does not mention whether it is possible to apply for just single course units, only that the application process for recognition of previous studies is available for degree courses.'

In [ ]:
q8 = "My Bachelor's degree was a double degree program obtained from two different universities, one from Indonesia and one from Singapore, how does this affect my application for admission for the master's degree programs?"
result = rag_chain.invoke(q8)
llm_outputs.append(result)# append the answer of your llm to the 8th question as a string here
result

' A Bachelor’s degree or equivalent foreign qualification with specific curricular requirements obtained from a higher education institution accredited according to the national educational accreditation system of the country where it is located.'

In [ ]:
q9 = "what are the eligibilities to apply for the admission to a master's degree program?"
result = rag_chain.invoke(q9)
llm_outputs.append(result)# append the answer of your llm to the 9th question as a string here
result

' To be eligible for admission to a Master’s degree, you need a Bachelor’s degree or equivalent foreign qualification with specific curricular requirements obtained from a higher education institution accredited according to the national educational accreditation system of the country where it is located.'

In [ ]:
q10 = "Can you give the contact info so I can get more detailed academic information?"
result = rag_chain.invoke(q10)
llm_outputs.append(result)# append the answer of your llm to the 10th question as a string here
result

' Write a letter addressed to the Admissions Office and include the following information:\n- Your full name\n- Your academic background\n- Your motivation for studying at the University of Padua\n- Any relevant documents\n\nApply for a single course unit\nAnswer: Write a letter addressed to the Admissions Office and include the following information:\n- Your full name\n- The course unit you are interested in attending\n- Your motivation for taking the course unit\n\nFor more detailed information, please contact the programme coordinator/secretariat.'

Make sure that your *llm_output* contains 10 strings(answers of your LLM) rspectively to the order that the questions asked



In [ ]:
# check the number of elements in array, it should be exactly 10
print(len(llm_outputs))

10


In [ ]:
# make sure the llm_outputs is filled in the same order that the questions asked
print(llm_outputs)

[' No, the context does not provide any information about the possibility of applying for the admission of the University of Padua for non-Italian students.', ' No, the context does not specify that the courses provided by the university of Padua are taught only in Italian.', ' No, the context does not allow you to enroll and submit the application for the second (winter) semester.', ' Applicants need to have a minimum of 12 years of schooling and pass all the scheduled exams to be eligible for admission to a Bachelor’s degree. Applicants can consult a country-based list of qualifications to verify their entry qualification.', " No, the context does not mention that the 11-year diploma means that the applicant cannot apply for the admission of the bachelor's degree program in biology of the University of Padua.", ' Applicants should check the country-based list of qualifications to verify if their entry qualification is suitable to access their degree programme of interest.', ' No, the

In [ ]:
# do not run this part
llm_outputs = [
    ' No, the context does not provide any information about the possibility of applying for the admission of the University of Padua for non-Italian students.',
    ' No, the context does not specify that the courses provided by the university of Padua are taught only in Italian.',
    ' No, the context does not allow you to enroll and submit the application for the second (winter) semester.',
    ' Applicants need to have a minimum of 12 years of schooling and pass all the scheduled exams to be eligible for admission to a Bachelor’s degree. Applicants can consult a country-based list of qualifications to verify their entry qualification.',
    " No, the context does not mention that the 11-year diploma means that the applicant cannot apply for the admission of the bachelor's degree program in biology of the University of Padua.",
    ' Applicants should check the country-based list of qualifications to verify if their entry qualification is suitable to access their degree programme of interest.',
    ' No, the context does not mention whether it is possible to apply for just single course units, only that the application process for recognition of previous studies is available for degree courses.',
    ' A Bachelor’s degree or equivalent foreign qualification with specific curricular requirements obtained from a higher education institution accredited according to the national educational accreditation system of the country where it is located.',
    ' To be eligible for admission to a Master’s degree, you need a Bachelor’s degree or equivalent foreign qualification with specific curricular requirements obtained from a higher education institution accredited according to the national educational accreditation system of the country where it is located.',
    ' Write a letter addressed to the Admissions Office and include the following information:\n- Your full name\n- Your academic background\n- Your motivation for studying at the University of Padua\n- Any relevant documents\n\nApply for a single course unit\nAnswer: Write a letter addressed to the Admissions Office and include the following information:\n- Your full name\n- The course unit you are interested in attending\n- Your motivation for taking the course unit\n\nFor more detailed information, please contact the programme coordinator/secretariat.']


The bellow is the expected answers that your LLM should give.

**Please do not change it at all so we all have unified measurement criteria**

Of course your LLM answers should not be exactly the same as bellow ones but should have the similar semantic

In [18]:
expected_answers = ["yes, you can. The University of Padua welcomes applications from international students. You are considered an “international student” if you hold a non-Italian academic diploma, degree or equivalent.",
                    "No. There are two type of courses; ENGLISH-taught degree courses and ITALIAN-taught degree courses ",
                    "No, unfortunately you can not. courses at the University of Padua start at the end of September/beginning of October. Enrolment for the 2nd semester is not available.",
                    "To be eligible for admission to a Bachelor’s degree or a single-cycle degree, you need a minimum 12-year school record. If you have been awarded a final diploma after a shorter programme of study (i.e. less than 12 years), in order to become eligible you must: have attended at least 1 (in case of 11 years of schooling) or 2 (in case of 10 years of schooling) years of academic studies and passed all the scheduled exams. OR have obtained a post-secondary qualification  in a non-University higher education institution, certifying the additional one or two years of education. OR have successfully completed one or two year preparatory courses (Foundation courses) to compensate for the missing years of schooling.",
                    "you must take additional courses to become eligible for admission to a Bachelor's degree program. If you have been awarded a final diploma after a shorter programme of study (i.e. less than 12 years), in order to become eligible you must: have attended at least 1 (in case of 11 years of schooling) or 2 (in case of 10 years of schooling) years of academic studies and passed all the scheduled exams. OR have obtained a post-secondary qualification  in a non-University higher education institution, certifying the additional one or two years of education. OR have successfully completed one or two year preparatory courses (Foundation courses) to compensate for the missing years of schooling. Or as an alternative, you must: have attended an additional one (in case of 12 years of schooling) or two (in case of 11 years of schooling) years of academic studies in another country and/or different foreign system and passed all the scheduled exams. OR have obtained an official Italian or foreign post-secondary qualification in a non-University higher education institution with a minimum duration of one or two years. OR have successfully completed Foundation courses held from an Italian University. These additional certificates only allow enrolment to the first year and cannot be further evaluated for shortening of your degree programme",
                    "You can check a country-based list of qualifications in order to verify whether their entry qualification is suitable to access your degree programme of interest.",
                    "yes, you can. If you are you interested in attending single course units and take the corresponding exams without having to be enrolled in any degree course, visit the dedicated page.",
                    "If your bachelor's degree was issued by two or more universities (e.g. the awarding and the teaching institutions are not the same and pertain to different national education systems), each institution must be accredited according to the national educational accreditation system of the country where it is located..",
                    "To be eligible for admission to a Master’s degree, you need a Bachelor’s degree or equivalent foreign qualification with specific curricular requirements obtained from a higher education institution accredited according to the national educational accreditation system of the country where it is located.",
                    "sure,For academic related information you can contact the programme coordinator/secretariat tel. +39 0498273131, or send an email to urp@unipd.it "]

Install the following library

**Note: it may ask you to restart the colab runtime after installation but its not necessary since in case of restartin you have to run the above code again**

In [17]:
!pip install -U deepeval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.6/245.6 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.0/289.0 kB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.1/46.1 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.1/86.1 kB 10.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.7/320.7 kB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 22.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.6/40.6 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.1/71.1 kB 19.6 MB/s eta 0:00:00
  Created wheel for docx2txt: filename=docx2txt-0.8-py3-none-any.whl size=3960 sha256=2cf604e33aa9bc1c9280fc3f6047d0642a3ef75ee85e91a4f3428f16fcbb24fe
  Stored in directory: /root/.cache/pip/wheels/22/58/cf/093d0a6c3ecfdfc5f6

In [12]:
import os
from getpass import getpass

Run the bellow code and pass the Open AI API Key you are provided.

**please save your api key somewhere in your computer where you can access to it whenever needed. The API Key can not be retrieved if you lose it.**

In [13]:
# Get the API token from the user
OPENAI_API_KEY = getpass("Enter your openai API token: ")

# Set the environment variable
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

Enter your openai API token: ··········


The following array is obtained from the unipd webpage and it will be passed to the GPT-4o as a source of judgement

**Please do not change its content or order**

In [14]:
retrieval_contexts = [
    "The University of Padua welcomes applications from international students. You are considered an “international student” if you hold a non-Italian academic diploma, degree or equivalent.",
    "There are two types of degree courses; ENGLISH-taught degree courses: check application procedures and the course factsheets for detailed information. In order to submit your application (except for the Single-cycle degree programme in Medicine and Surgery), please go to: apply.unipd.it ITALIAN-taught degree courses: applications dates vary depending on the course - check application procedures",
    "Courses at the University of Padua start at the end of September/beginning of October. Enrolment for the 2nd semester is not available.",
    "To be eligible for admission to a Bachelor’s degree or a single-cycle degree, you need a minimum 12-year school record. If you have been awarded a final diploma after a shorter programme of study (i.e. less than 12 years), in order to become eligible you must: have attended at least 1 (in case of 11 years of schooling) or 2 (in case of 10 years of schooling) years of academic studies and passed all the scheduled exams. OR have obtained a post-secondary qualification  in a non-University higher education institution, certifying the additional one or two years of education. OR have successfully completed one or two year preparatory courses (Foundation courses) to compensate for the missing years of schooling.",
    "To be eligible for admission to a Bachelor’s degree or a single-cycle degree, you need a minimum 12-year school record. If you have been awarded a final diploma after a shorter programme of study (i.e. less than 12 years), in order to become eligible you must: have attended at least 1 (in case of 11 years of schooling) or 2 (in case of 10 years of schooling) years of academic studies and passed all the scheduled exams. OR have obtained a post-secondary qualification  in a non-University higher education institution, certifying the additional one or two years of education. OR have successfully completed one or two year preparatory courses (Foundation courses) to compensate for the missing years of schooling. As an alternative to this certification, you must: have attended an additional one (in case of 12 years of schooling) or two (in case of 11 years of schooling) years of academic studies in another country and/or different foreign system and passed all the scheduled exams. OR have obtained an official Italian or foreign post-secondary qualification in a non-University higher education institution with a minimum duration of one or two years. OR have successfully completed Foundation courses held from an Italian University. These additional certificates only allow enrolment to the first year and cannot be further evaluated for shortening of your degree programme",
    "Foreign entry qualifications suitable for admission; Applicants can consult a country-based list of qualifications in order to verify whether their entry qualification is suitable to access their degree programme of interest..",
    "Apply for a single course unit ; If you are you interested in attending single course units and take the corresponding exams without having to be enrolled in any degree course, visit the dedicated page.",
    "Should your study title be issued by two or more universities (e.g. the awarding and the teaching institutions are not the same and pertain to different national education systems), each institution must be accredited according to the national educational accreditation system of the country where it is located.",
    "To be eligible for admission to a Master’s degree, you need a Bachelor’s degree or equivalent foreign qualification with specific curricular requirements obtained from a higher education institution accredited according to the national educational accreditation system of the country where it is located.",
    "Global Engagement Office - Recruitment Unit Palazzo Anselmi Lungargine del Piovego 1 - 35131 Padova Get in touch with the Global Engagement Office For academic related information, please contact the programme coordinator/secretariat tel. +39 0498273131 or send an email to  urp@unipd.it "
]


In [15]:
from deepeval import evaluate
from deepeval.metrics.ragas import RagasMetric
from deepeval.test_case import LLMTestCase

In [19]:
actual_outputs = llm_outputs

expected_outputs = expected_answers

test_cases = []
# Do not change the threshold or model for the sake of having similar measurement criteria
metric = RagasMetric(threshold=0.6, model="gpt-4o")

# Loop through each question and create a test case
for idx, (actual_output, expected_output, retrieval_context) in enumerate(zip(actual_outputs, expected_outputs, retrieval_contexts)):
    test_case = LLMTestCase(
        input=f"Question {idx + 1}",
        actual_output=actual_output,
        expected_output=expected_output,
        retrieval_context=[retrieval_context]  # Wrap the string in a list
    )
    test_cases.append(test_case)

# Evaluate test cases in batch
metrics = evaluate(test_cases, [metric])


Evaluating test cases...
Event loop is already running. Applying nest_asyncio patch to allow async execution...


Overall Metric Pass Rates





No test cases found, please try again.

# Final Step:
After successfully running the above tests, run the following and save the results and send it into our group



In [20]:
from statistics import mean

# puting the results of metrics in test_results to avoid confusion
test_results = metrics
# Initialize lists to collect scores for each metric and overall scores
contextual_precision_scores = []
contextual_recall_scores = []
faithfulness_scores = []
answer_relevancy_scores = []
overall_scores = []

# Iterate through each test result to extract the scores from score_breakdown and overall score
for test_result in test_results:
    # Access the RagasMetric object
    ragas_metric = test_result.metrics[0]

    # Access the score_breakdown dictionary
    score_breakdown = ragas_metric.score_breakdown

    # Append the scores to the respective lists
    contextual_precision_scores.append(score_breakdown['Contextual Precision (ragas)'])
    contextual_recall_scores.append(score_breakdown['Contextual Recall (ragas)'])
    faithfulness_scores.append(score_breakdown['Faithfulness (ragas)'])
    answer_relevancy_scores.append(score_breakdown['Answer Relevancy (ragas)'])

    # Append the overall score to the overall_scores list
    overall_scores.append(ragas_metric.score)

# Calculate the average for each metric
avg_contextual_precision = mean(contextual_precision_scores)
avg_contextual_recall = mean(contextual_recall_scores)
avg_faithfulness = mean(faithfulness_scores)
avg_answer_relevancy = mean(answer_relevancy_scores)
avg_overall_score = mean(overall_scores)

# Print the average scores

print("Gemma 2B results:")
print(f"Contextual Precision (average score: {avg_contextual_precision})")
print(f"Contextual Recall (average score: {avg_contextual_recall})")
print(f"Faithfulness (average score: {avg_faithfulness})")
print(f"Answer Relevancy (average score: {avg_answer_relevancy})")
print(f"RAGAS (average overall score: {avg_overall_score})")

StatisticsError: mean requires at least one data point

**Note: if you are working on two different sub models(e.g. I worked on Llama3 8b and lama3 70b), you should run all above code once again for the other sub model**

##<============================================================================Gemma7B===============================================================================>

In [21]:
from langchain_community.llms import HuggingFaceEndpoint
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

# Define the repository ID for the Gemma 7b model
repo_id = "google/gemma-7b-it"

# Set up a Hugging Face Endpoint for Gemma 7b model
llm = HuggingFaceEndpoint(
    repo_id=repo_id, max_length=1024, temperature=0.1
)

                    max_length was transferred to model_kwargs.
                    Please make sure that max_length is what you intended.


The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [22]:
# Load a document
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader(Rag_input_data)
data = loader.load()
#print(data)

In [23]:
from langchain_community.document_loaders import TextLoader
from langchain_community.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings,
)
from langchain_community.vectorstores import Chroma
from langchain_text_splitters import CharacterTextSplitter


# split it into chunks
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(data)

# create the open-source embedding function
embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

# load it into Chroma
db = Chroma.from_documents(docs, embedding_function)

In [24]:
from langchain import hub
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain.chains import RetrievalQA

retriever = db.as_retriever(search_type="mmr", search_kwargs={'k': 4, 'fetch_k': 20})
prompt = hub.pull("rlm/rag-prompt")

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
)

In [25]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

tokenizer = AutoTokenizer.from_pretrained("google/gemma-7b-it")
model = AutoModelForCausalLM.from_pretrained(
    "google/gemma-7b-it",
    torch_dtype=torch.bfloat16
)

input_text = "Write me a poem about Machine Learning."
input_ids = tokenizer(input_text, return_tensors="pt")

tokenizer_config.json:   0%|          | 0.00/34.2k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/694 [00:00<?, ?B/s]

...

model-00002-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/2.11G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

In [27]:
q1= "Can I apply for the admission of the university of Padua if I am a non-italian student?"

# Ask the question from your LLM and then append its output to the below array

llm_outputs = []
result = rag_chain.invoke(q1)
llm_outputs.append(result)# append the answer of your llm to the 1st question as a string here
result

HfHubHTTPError:  (Request ID: 2CSCU-N35_XyaMUqZp05y)

403 Forbidden: None.
Cannot access content at: https://api-inference.huggingface.co/models/google/gemma-7b-it.
If you are trying to create or update content,make sure you have a token with the `write` role.
The model google/gemma-7b-it is too large to be loaded automatically (17GB > 10GB). Please use Spaces (https://huggingface.co/spaces) or Inference Endpoints (https://huggingface.co/inference-endpoints).

In [ ]:
q2 = "Are the courses provided by the university of Padua taught only in Italian?"
result = rag_chain.invoke(q2)
llm_outputs.append(result)# append the answer of your llm to the 2n question as a string here
result

' No, the context does not specify that the courses provided by the university of Padua are taught only in Italian.'

In [ ]:
q3 = "I missed the deadlines for submitting application for the first semester. Can I enroll and submit the application for the second (winter) semester?"
result = rag_chain.invoke(q3)
llm_outputs.append(result)# append the answer of your llm to the 3rd question as a string here
result

' No, the context does not allow you to enroll and submit the application for the second (winter) semester.'

In [ ]:
q4 = "I want to apply for the bachelor's degree in biology, how to know if I am eligible?"
result = rag_chain.invoke(q4)
llm_outputs.append(result)# append the answer of your llm to the 4th question as a string here
result

' Applicants need to have a minimum of 12 years of schooling and pass all the scheduled exams to be eligible for admission to a Bachelor’s degree. Applicants can consult a country-based list of qualifications to verify their entry qualification.'

In [ ]:
q5 = "The duration of diploma that I obtained is 11 years in my country, does that mean that I can not apply for the admission of the bachelor's degree program in biology of the university of Padua?"
result = rag_chain.invoke(q5)
llm_outputs.append(result)# append the answer of your llm to the 5th question as a string here
result

" No, the context does not mention that the 11-year diploma means that the applicant cannot apply for the admission of the bachelor's degree program in biology of the University of Padua."

In [ ]:
q6 ="How can I make sure if my previous university which is not Italian is qualified in order to apply for the master's degree programs?"
result = rag_chain.invoke(q6)
llm_outputs.append(result)# append the answer of your llm to the 6th question as a string here
result

' Applicants should check the country-based list of qualifications to verify if their entry qualification is suitable to access their degree programme of interest.'

In [ ]:
q7 = "Can I apply for just single course units not the whole degree programs of the University of Padua?"
result = rag_chain.invoke(q7)
llm_outputs.append(result)# append the answer of your llm to the 7th question as a string here
result

' No, the context does not mention whether it is possible to apply for just single course units, only that the application process for recognition of previous studies is available for degree courses.'

In [ ]:
q8 = "My Bachelor's degree was a double degree program obtained from two different universities, one from Indonesia and one from Singapore, how does this affect my application for admission for the master's degree programs?"
result = rag_chain.invoke(q8)
llm_outputs.append(result)# append the answer of your llm to the 8th question as a string here
result

' A Bachelor’s degree or equivalent foreign qualification with specific curricular requirements obtained from a higher education institution accredited according to the national educational accreditation system of the country where it is located.'

In [ ]:
q9 = "what are the eligibilities to apply for the admission to a master's degree program?"
result = rag_chain.invoke(q9)
llm_outputs.append(result)# append the answer of your llm to the 9th question as a string here
result

' To be eligible for admission to a Master’s degree, you need a Bachelor’s degree or equivalent foreign qualification with specific curricular requirements obtained from a higher education institution accredited according to the national educational accreditation system of the country where it is located.'

In [ ]:
q10 = "Can you give the contact info so I can get more detailed academic information?"
result = rag_chain.invoke(q10)
llm_outputs.append(result)# append the answer of your llm to the 10th question as a string here
result

' Write a letter addressed to the Admissions Office and include the following information:\n- Your full name\n- Your academic background\n- Your motivation for studying at the University of Padua\n- Any relevant documents\n\nApply for a single course unit\nAnswer: Write a letter addressed to the Admissions Office and include the following information:\n- Your full name\n- The course unit you are interested in attending\n- Your motivation for taking the course unit\n\nFor more detailed information, please contact the programme coordinator/secretariat.'

Make sure that your *llm_output* contains 10 strings(answers of your LLM) rspectively to the order that the questions asked



In [ ]:
# check the number of elements in array, it should be exactly 10
print(len(llm_outputs))

10


In [ ]:
# make sure the llm_outputs is filled in the same order that the questions asked
print(llm_outputs)

[' No, the context does not provide any information about the possibility of applying for the admission of the University of Padua for non-Italian students.', ' No, the context does not specify that the courses provided by the university of Padua are taught only in Italian.', ' No, the context does not allow you to enroll and submit the application for the second (winter) semester.', ' Applicants need to have a minimum of 12 years of schooling and pass all the scheduled exams to be eligible for admission to a Bachelor’s degree. Applicants can consult a country-based list of qualifications to verify their entry qualification.', " No, the context does not mention that the 11-year diploma means that the applicant cannot apply for the admission of the bachelor's degree program in biology of the University of Padua.", ' Applicants should check the country-based list of qualifications to verify if their entry qualification is suitable to access their degree programme of interest.', ' No, the

The bellow is the expected answers that your LLM should give.

**Please do not change it at all so we all have unified measurement criteria**

Of course your LLM answers should not be exactly the same as bellow ones but should have the similar semantic

In [ ]:
expected_answers = ["yes, you can. The University of Padua welcomes applications from international students. You are considered an “international student” if you hold a non-Italian academic diploma, degree or equivalent.",
                    "No. There are two type of courses; ENGLISH-taught degree courses and ITALIAN-taught degree courses ",
                    "No, unfortunately you can not. courses at the University of Padua start at the end of September/beginning of October. Enrolment for the 2nd semester is not available.",
                    "To be eligible for admission to a Bachelor’s degree or a single-cycle degree, you need a minimum 12-year school record. If you have been awarded a final diploma after a shorter programme of study (i.e. less than 12 years), in order to become eligible you must: have attended at least 1 (in case of 11 years of schooling) or 2 (in case of 10 years of schooling) years of academic studies and passed all the scheduled exams. OR have obtained a post-secondary qualification  in a non-University higher education institution, certifying the additional one or two years of education. OR have successfully completed one or two year preparatory courses (Foundation courses) to compensate for the missing years of schooling.",
                    "you must take additional courses to become eligible for admission to a Bachelor's degree program. If you have been awarded a final diploma after a shorter programme of study (i.e. less than 12 years), in order to become eligible you must: have attended at least 1 (in case of 11 years of schooling) or 2 (in case of 10 years of schooling) years of academic studies and passed all the scheduled exams. OR have obtained a post-secondary qualification  in a non-University higher education institution, certifying the additional one or two years of education. OR have successfully completed one or two year preparatory courses (Foundation courses) to compensate for the missing years of schooling. Or as an alternative, you must: have attended an additional one (in case of 12 years of schooling) or two (in case of 11 years of schooling) years of academic studies in another country and/or different foreign system and passed all the scheduled exams. OR have obtained an official Italian or foreign post-secondary qualification in a non-University higher education institution with a minimum duration of one or two years. OR have successfully completed Foundation courses held from an Italian University. These additional certificates only allow enrolment to the first year and cannot be further evaluated for shortening of your degree programme",
                    "You can check a country-based list of qualifications in order to verify whether their entry qualification is suitable to access your degree programme of interest.",
                    "yes, you can. If you are you interested in attending single course units and take the corresponding exams without having to be enrolled in any degree course, visit the dedicated page.",
                    "If your bachelor's degree was issued by two or more universities (e.g. the awarding and the teaching institutions are not the same and pertain to different national education systems), each institution must be accredited according to the national educational accreditation system of the country where it is located..",
                    "To be eligible for admission to a Master’s degree, you need a Bachelor’s degree or equivalent foreign qualification with specific curricular requirements obtained from a higher education institution accredited according to the national educational accreditation system of the country where it is located.",
                    "sure,For academic related information you can contact the programme coordinator/secretariat tel. +39 0498273131, or send an email to urp@unipd.it "]

In [ ]:
from deepeval import evaluate
from deepeval.metrics.ragas import RagasMetric
from deepeval.test_case import LLMTestCase

In [ ]:
actual_outputs = llm_outputs

expected_outputs = expected_answers

test_cases = []
# Do not change the threshold or model for the sake of having similar measurement criteria
metric = RagasMetric(threshold=0.6, model="gpt-4o")

# Loop through each question and create a test case
for idx, (actual_output, expected_output, retrieval_context) in enumerate(zip(actual_outputs, expected_outputs, retrieval_contexts)):
    test_case = LLMTestCase(
        input=f"Question {idx + 1}",
        actual_output=actual_output,
        expected_output=expected_output,
        retrieval_context=[retrieval_context]  # Wrap the string in a list
    )
    test_cases.append(test_case)

# Evaluate test cases in batch
metrics = evaluate(test_cases, [metric])


Output()

Evaluating test cases...
Event loop is already running. Applying nest_asyncio patch to allow async execution...


Evaluating:   0%|          | 0/1 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/1 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/1 [00:00<?, ?it/s]

...

Evaluating:   0%|          | 0/1 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/1 [00:00<?, ?it/s]



Metrics Summary

  - ❌ RAGAS (score: 0.4166666666416666, threshold: 0.6, strict: False, evaluation model: gpt-4o, reason: None, error: None)
      - Contextual Precision (ragas) (score: 0.9999999999)
      - Contextual Recall (ragas) (score: 0.6666666666666666)
      - Faithfulness (ragas) (score: 0.0)
      - Answer Relevancy (ragas) (score: 0.0)

For test case:

  - input: Question 1
  - actual output:  No, the context does not provide any information about the possibility of applying for the admission of the University of Padua for non-Italian students.
  - expected output: yes, you can. The University of Padua welcomes applications from international students. You are considered an “international student” if you hold a non-Italian academic diploma, degree or equivalent.
  - context: None
  - retrieval context: ['The University of Padua welcomes applications from international students. You are considered an “international student” if you hold a non-Italian academic diploma, degre

/usr/local/lib/python3.10/dist-packages/portalocker/utils.py:218: UserWarning: timeout has no effect in blocking mode
  warnings.warn(


✅ Tests finished! Run "deepeval login" to view evaluation results on the web.

# Final Step:
After successfully running the above tests, run the following and save the results and send it into our group



In [ ]:
from statistics import mean

# puting the results of metrics in test_results to avoid confusion
test_results = metrics
# Initialize lists to collect scores for each metric and overall scores
contextual_precision_scores = []
contextual_recall_scores = []
faithfulness_scores = []
answer_relevancy_scores = []
overall_scores = []

# Iterate through each test result to extract the scores from score_breakdown and overall score
for test_result in test_results:
    # Access the RagasMetric object
    ragas_metric = test_result.metrics[0]

    # Access the score_breakdown dictionary
    score_breakdown = ragas_metric.score_breakdown

    # Append the scores to the respective lists
    contextual_precision_scores.append(score_breakdown['Contextual Precision (ragas)'])
    contextual_recall_scores.append(score_breakdown['Contextual Recall (ragas)'])
    faithfulness_scores.append(score_breakdown['Faithfulness (ragas)'])
    answer_relevancy_scores.append(score_breakdown['Answer Relevancy (ragas)'])

    # Append the overall score to the overall_scores list
    overall_scores.append(ragas_metric.score)

# Calculate the average for each metric
avg_contextual_precision = mean(contextual_precision_scores)
avg_contextual_recall = mean(contextual_recall_scores)
avg_faithfulness = mean(faithfulness_scores)
avg_answer_relevancy = mean(answer_relevancy_scores)
avg_overall_score = mean(overall_scores)

# Print the average scores

print("Gemma 2B results:")
print(f"Contextual Precision (average score: {avg_contextual_precision})")
print(f"Contextual Recall (average score: {avg_contextual_recall})")
print(f"Faithfulness (average score: {avg_faithfulness})")
print(f"Answer Relevancy (average score: {avg_answer_relevancy})")
print(f"RAGAS (average overall score: {avg_overall_score})")

Llama3 70b results:
Contextual Precision (average score: 0.9999999999)
Contextual Recall (average score: 0.8333333333333334)
Faithfulness (average score: 0.6433333333333333)
Answer Relevancy (average score: 0.6807886884420109)
RAGAS (average overall score: 0.7893638387521694)
